In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

from sys import path
from os.path import abspath
path.append(abspath("/home/ng136/nico"))
import ng_functions as ng

from glob import glob
from pathlib import Path
import warnings
from time import time
from datetime import datetime
from tqdm import tqdm

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
print(datetime.today())

2022-12-15 22:54:35.619100


# I/O

# Load annotated experiments 2 and 3 (neutrophils only)

In [3]:
n_states = ['N1a', 'N1b', 'N2', 'N3', 'N4', 'N5', 'N6']


In [4]:
n_states_dict = {'N1a' : 'Sell+Ngp+ neutrophils',
                 'N1b' : 'Sell+Lst1+ neutrophils',
                 'N2' : 'Sell+Cxcl10+ neutrophils',
                 'N3' : 'Cxcl3+ neutrophils',
                 'N4' : 'Siglecf+Xbp1+ neutrophils',
                 'N5' : 'Siglecf+Ccl3+ neutrophils',
                 'N6' : 'Siglecf+Ngp+ neutrophils'}

neu_palette_dict = {'Sell+Ngp+ neutrophils': '#6EA6CD',
                    'Sell+Lst1+ neutrophils': '#4A7BB7',
                    'Sell+Cxcl10+ neutrophils': '#364B9A',
                    'Cxcl3+ neutrophils': '#EAECCC',
                    'Siglecf+Xbp1+ neutrophils': '#A50026',
                    'Siglecf+Ccl3+ neutrophils': '#DD3D2D',
                    'Siglecf+Ngp+ neutrophils': '#F67E4B'}

In [5]:
ndata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_neutrophils_untreated_annotated_embedding_5258x13126_backup_220422_15h46.h5ad')
tdata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_neutrophils_acd40_annotated_embedding_25210x17824_backup_220422_15h46.h5ad')

ndata.obs['cell_state_annotation'] = ndata.obs['smoothed_Zilionis']
tdata.obs['cell_state_annotation'] = tdata.obs['smoothed_Zilionis']

hdata = ndata[ndata.obs.sample_condition.str.startswith('WT')].copy()
udata = ndata[ndata.obs.sample_condition.str.startswith('KP19')].copy()


# ADT

In [6]:
nadtdata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_neutrophils_untreated_adt_clr_no_cd101_3978x20_backup_220601_04h57.h5ad')
tadtdata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_neutrophils_treated_adt_clr_25210x20_backup_220601_05h05.h5ad')

nadtdata.obs['cell_state_annotation'] = nadtdata.obs['smoothed_Zilionis']
tadtdata.obs['cell_state_annotation'] = tadtdata.obs['smoothed_Zilionis']


hadtdata = nadtdata[nadtdata.obs.sample_condition.str.startswith('WT')].copy()
uadtdata = nadtdata[nadtdata.obs.sample_condition.str.startswith('KP19')].copy()

# SPRING

# Set up X 

In [7]:
ndata.X = ndata.layers['cp10k']
hdata.X = hdata.layers['cp10k']
udata.X = udata.layers['cp10k']
tdata.X = tdata.layers['cp10k']

nadtdata.X = nadtdata.layers['clr']
hadtdata.X = hadtdata.layers['clr']
uadtdata.X = uadtdata.layers['clr']
tadtdata.X = tadtdata.layers['clr']

# Set up obs

In [8]:
for adata in [ndata, hdata, udata, tdata]:
    adata.obs = adata.obs.loc[:,adata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
                                                        'library_name', 'sample_condition', 
                                                        'cell_state_annotation'])]
    
    
for adtdata in [nadtdata, hadtdata, uadtdata]:
    adtdata.obs = adtdata.obs.loc[:,adtdata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
                                                    'library_name', 'sample_condition', 'CD101_Ab', 'Lair1_Ab',
                                                    'cell_state_annotation'])]


tadtdata.obs = tadtdata.obs.loc[:,tadtdata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
                                                    'library_name', 'sample_condition', 'Rat_IgG2a_Ab', 'cell_state_annotation'])]
    
    



# ndata.obs = ndata.obs.loc[:,ndata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
#                                                     'library_name', 'sample_condition', 'most_likely_Zilionis', 'smoothed_Zilionis'])]

# tdata.obs = tdata.obs.loc[:,tdata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
#                                                     'library_name', 'sample_condition', 'most_likely_Zilionis', 
#                                                     'smoothed_Zilionis', 'smoothed_Zilionis_split', 'smoothed_Zilionis_sub'])]


# nadtdata.obs = nadtdata.obs.loc[:,nadtdata.obs.columns.isin(['n_counts', 'n_genes', 'mito_fraction', 
#                                                     'library_name', 'sample_condition', 'CD101_Ab', 'Lair1_Ab',
#                                                     'most_likely_Zilionis', 'smoothed_Zilionis'])]




In [9]:
for adata in [ndata, hdata, udata, tdata, nadtdata, hadtdata, uadtdata, tadtdata]:
    adata.obs['neutrophil_subset'] = adata.obs['cell_state_annotation'].map(n_states_dict).astype('category')
    adata.obs['neutrophil_subset'] = adata.obs['neutrophil_subset'].cat.reorder_categories(sorted(adata.obs['neutrophil_subset'].cat.categories))
    
    adata.uns['neutrophil_subset_colors'] = adata.obs['neutrophil_subset'].cat.categories.map(neu_palette_dict)

# Save colors

In [10]:
# neu_palette = ['#6EA6CD', '#4A7BB7', '#364B9A', '#EAECCC', '#A50026', '#DD3D2D', '#F67E4B']
# neu_palette_reduced = ['#6EA6CD', '#4A7BB7', '#364B9A', '#A50026', '#DD3D2D', '#F67E4B'] # no N3 color


# for adata in [ndata, udata, tdata, nadtdata, uadtdata, tadtdata]:
#     adata.uns['neutrophil_subset_colors'] = neu_palette

# hdata.uns['neutrophil_subset_colors'] = neu_palette_reduced
# hadtdata.uns['neutrophil_subset_colors'] = neu_palette_reduced

    
    


# # ndata.uns['neutrophil_subset'] = ndata.uns['smoothed_Zilionis_colors'] = neu_palette
# # hdata.uns['neutrophil_subset'] = hdata.uns['smoothed_Zilionis_colors'] = neu_palette_reduced
# # udata.uns['most_likely_Zilionis_colors'] = udata.uns['smoothed_Zilionis_colors'] = neu_palette
# # tdata.uns['most_likely_Zilionis_colors'] = tdata.uns['smoothed_Zilionis_colors'] = neu_palette

# # nadtdata.uns['most_likely_Zilionis_colors'] = nadtdata.uns['smoothed_Zilionis_colors'] = neu_palette
# # hadtdata.uns['most_likely_Zilionis_colors'] = hadtdata.uns['smoothed_Zilionis_colors'] = neu_palette_reduced
# # uadtdata.uns['most_likely_Zilionis_colors'] = uadtdata.uns['smoothed_Zilionis_colors'] = neu_palette
# # tadtdata.uns['most_likely_Zilionis_colors'] = tadtdata.uns['smoothed_Zilionis_colors'] = neu_palette

# Set up UMAP

In [11]:
# nadtdata.obsm['X_umap'] = ndata[ndata.obs.sample_condition != 'KP19_CD101'].obsm['X_umap']
# hadtdata.obsm['X_umap'] = hdata[hdata.obs.sample_condition != 'KP19_CD101'].obsm['X_umap']
# uadtdata.obsm['X_umap'] = udata[udata.obs.sample_condition != 'KP19_CD101'].obsm['X_umap']
# tadtdata.obsm['X_umap'] = tdata.obsm['X_umap']

# nadtdata.obsp =  ndata[ndata.obs.sample_condition != 'KP19_CD101'].obsp
# hadtdata.obsp =  hdata[hdata.obs.sample_condition != 'KP19_CD101'].obsp
# uadtdata.obsp =  udata[udata.obs.sample_condition != 'KP19_CD101'].obsp
# tadtdata.obsp =  tdata.obsp

# nadtdata.uns['neighbors'] = ndata.uns['neighbors']
# hadtdata.uns['neighbors'] = hdata.uns['neighbors']
# uadtdata.uns['neighbors'] = udata.uns['neighbors']
# tadtdata.uns['neighbors'] = tdata.uns['neighbors']

# SAVE SPRING

In [23]:
ng.save_spring(ndata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_untreated_gex', embedding_name='gex')
ng.save_spring(hdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_healthy_gex', embedding_name='gex')
ng.save_spring(udata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_kp19_gex', embedding_name='gex')
ng.save_spring(tdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_acd40_gex', embedding_name='gex')

Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_untreated_gex/gex


/n/groups/klein/nico/neutrophils/analysis/conda-env/lib/python3.8/site-packages/scanpy/external/exporting.py:151: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(adata.obs[obs_name]):
/n/groups/klein/nico/neutrophils/analysis/conda-env/lib/python3.8/site-packages/scanpy/external/exporting.py:167: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(adata.obs[obs_name]):


Elapsed time: 0 minutes and 19.9 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_healthy_gex/gex
Elapsed time: 0 minutes and 13.9 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_kp19_gex/gex
Elapsed time: 0 minutes and 18.1 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_acd40_gex/gex
Elapsed time: 1 minutes and 6.7 seconds.


In [24]:
ng.save_spring(nadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_untreated_adt', embedding_name='adt')
ng.save_spring(hadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_healthy_adt', embedding_name='adt')
ng.save_spring(uadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_kp19_adt', embedding_name='adt')
ng.save_spring(tadtdata, '/n/groups/klein/nico/neutrophils/spring/221215/', sample_name='neutrophils_acd40_adt', embedding_name='adt')

Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_untreated_adt/adt
Elapsed time: 0 minutes and 3.6 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_healthy_adt/adt
Elapsed time: 0 minutes and 1.4 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_kp19_adt/adt
Elapsed time: 0 minutes and 2.3 seconds.
Writing subplot to /n/groups/klein/nico/neutrophils/spring/221215/neutrophils_acd40_adt/adt
Elapsed time: 0 minutes and 22.9 seconds.


# Copy to SPRING directory

In [14]:
import shutil
import glob

In [26]:
spring_projects = glob.glob('/n/groups/klein/nico/neutrophils/spring/221215/*')

In [33]:
spring_projects

['/n/groups/klein/nico/neutrophils/spring/221215/all_cells_kp19_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_kp19_adt',
 '/n/groups/klein/nico/neutrophils/spring/221215/all_cells_untreated_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/all_cells_acd40_adt',
 '/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_healthy_adt',
 '/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_healthy_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_acd40_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_untreated_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_kp19_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/all_cells_kp19_adt',
 '/n/groups/klein/nico/neutrophils/spring/221215/all_cells_untreated_adt',
 '/n/groups/klein/nico/neutrophils/spring/221215/all_cells_acd40_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/all_cells_healthy_gex',
 '/n/groups/klein/nico/neutrophils/spring/221215/ne

In [37]:
for spring in spring_projects:
    shutil.copytree(spring, '/n/groups/kleintools/SPRING_private/gungabeesoon22/' + spring.replace('/n/groups/klein/nico/neutrophils/spring/',""))


In [32]:
spring

'/n/groups/klein/nico/neutrophils/spring/221215/neutrophils_acd40_adt'

'221215/neutrophils_acd40_adt'